In [1]:
import numpy
import tensorflow as tf

from model_builder import model_builder, relabel, class_merger, balancer

Load chips

In [2]:
specs = {
    'tensor': (
        '../urbangrammar_samba/'
        'spatial_signatures/chips/sample.npz'
    ),
}

In [3]:
data = numpy.load(specs["tensor"], allow_pickle=True)

In [4]:
chips = data["chips"]
labels = data["labels"]

Merge groups

In [5]:
group_mapping = [
    ['9_0', '9_1', '9_2', '9_4', '9_5', '2_0', '2_1', '2_2'],
    ['1_0', '3_0', '5_0', '6_0', '8_0'],
    ['0_0', '4_0', '7_0']
]
labels = class_merger(labels, group_mapping)

# If you don't want to merge groups, use `relabel` to ensure your labels are in the proper format
# labels = relabel(labels)

Drop chips from excessive classes

In [6]:
mask = balancer(labels, max_ratio=10)

Total number of selected chips: 18841 out of 65042
Counts:
 {0: 907, 1: 8864, 2: 9070}


In [7]:
labels = labels[mask]
chips = chips[mask]

Rescale

In [8]:
chips = chips / (chips.max() / 255.0)

Shuffle data to ensure full randomness before splitting to test, validation and secret.

In [20]:
numpy.random.seed(42)

shuffled_idx = numpy.arange(0, chips.shape[0])
numpy.random.shuffle(shuffled_idx)

chips = chips[shuffled_idx]
labels = labels[shuffled_idx]

Split data into train and validation.

In [24]:
batch_size = 32
split = int(chips.shape[0] * 0.8)

# assert that all labels are present in train and validation datasets
assert (numpy.unique(labels[:split]) == numpy.unique(labels[split:])).all()

train_dataset = tf.data.Dataset.from_tensor_slices((chips[:split], labels[:split]))
test_dataset = tf.data.Dataset.from_tensor_slices((chips[split:], labels[split:]))

train_dataset = train_dataset.batch(batch_size=batch_size)
test_dataset = test_dataset.batch(batch_size=batch_size)

Create model

In [52]:
model = model_builder(
    model_name="efficientnet", 
    bridge="flatten", 
    top_layer_neurons=512,
    n_labels=3,
)

In [53]:
model.summary()

Model: "efficientnet_flatten_512_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
resizing_10 (Resizing)       (None, 224, 224, 3)       0         
_________________________________________________________________
efficientnetb4 (Functional)  (None, 7, 7, 1792)        17673823  
_________________________________________________________________
flatten_10 (Flatten)         (None, 87808)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 512)               44958208  
_________________________________________________________________
dense_21 (Dense)             (None, 3)                 1539      
Total params: 62,633,570
Trainable params: 44,959,747
Non-trainable params: 17,673,823
___________________

Fit

In [54]:
epochs = 10
history = model.fit(train_dataset, epochs=epochs, validation_data=test_dataset)

Epoch 1/10
 56/471 [==>...........................] - ETA: 43s - loss: 12.4995 - accuracy: 0.5625

KeyboardInterrupt: 